In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=40, remove_stop_words=False)
train_pack_processed = preprocessor.fit_transform(train_pack_raw, drop_invalid=True)
dev_pack_processed = preprocessor.transform(dev_pack_raw, drop_invalid=False)
test_pack_processed = preprocessor.transform(test_pack_raw, drop_invalid=False)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3796.68it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2524.74it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1848057.36it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 4027.88it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:07<00:00, 2473.19it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 36116.35it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 2703.97it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [3]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f7104a60b70>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f7105ef4198>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'input_shapes': [(10,), (40,)]}

In [4]:
model = mz.models.ConvKNRM()
model.params.update(preprocessor.context)
model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['filters'] = 128 
model.params['conv_activation_func'] = 'tanh' 
model.params['max_ngram'] = 3
model.params['use_crossmatch'] = True 
model.params['kernel_num'] = 11
model.params['sigma'] = 0.1
model.params['exact_sigma'] = 0.001
model.params['optimizer'] = 'adadelta'
model.build()
model.compile()
#model.backend.summary()

In [5]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])
model.load_embedding_matrix(embedding_matrix)

In [6]:
pred_x, pred_y = test_pack_processed.unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=pred_x, y=pred_y, batch_size=len(pred_y))

In [7]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 255


In [8]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
255/255 [==============================] - 46s 181ms/step - loss: 0.4429
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.575526042026109 - normalized_discounted_cumulative_gain@5(0.0): 0.6453465201831717 - mean_average_precision(0.0): 0.5932570453764658
Epoch 2/30
255/255 [==============================] - 50s 197ms/step - loss: 0.1203
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6116090709087145 - normalized_discounted_cumulative_gain@5(0.0): 0.6603143733847261 - mean_average_precision(0.0): 0.6190537404769583
Epoch 3/30
255/255 [==============================] - 49s 193ms/step - loss: 0.0545
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6001499877858704 - normalized_discounted_cumulative_gain@5(0.0): 0.6660433061547467 - mean_average_precision(0.0): 0.6150657239348112
Epoch 4/30
255/255 [==============================] - 52s 203ms/step - loss: 0.0432
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6119184188197445 - norm

Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6610733889155886 - normalized_discounted_cumulative_gain@5(0.0): 0.7059824447926156 - mean_average_precision(0.0): 0.6585190244233725
